In [2]:
import pandas as pd

In [ ]:
filepath = '../data/total_factor_productivity.csv'

# Try reading with utf-8, fallback to latin1 if UnicodeDecodeError occurs
try:
	disp_df = pd.read_csv(filepath)
except UnicodeDecodeError:
	disp_df = pd.read_csv(filepath, encoding='latin1')

In [7]:
disp_df.head()

,naics4,year,sd,d7525,d9010,d9990,d1001,sd*,d7525*,d9010*,d9990*,d1001*
0,3111,1987.0,0.2883,0.2239,0.6477,0.6196,0.1455,0.2789,0.1947,0.4894,0.8043,0.1535
1,3111,1988.0,0.2782,0.1996,0.5330,0.8425,0.1384,0.3102,0.1529,0.5752,0.8945,0.1245
2,3111,1989.0,0.3253,0.1840,0.7220,0.7709,0.1533,0.3094,0.1547,0.5261,0.8397,0.1173
3,3111,1990.0,0.2954,0.2001,0.5154,1.037,0.1929,0.2927,0.1822,0.4714,0.7978,0.1004
4,3111,1991.0,0.2842,0.2235,0.5372,0.6796,0.2426,0.2842,0.1806,0.4711,0.7504,0.2352
